# Random Forest Bench


In [1]:
import cudf
import cuml
import sklearn as skl 
from cuml import RandomForestClassifier as cuRF
from sklearn.ensemble import RandomForestClassifier as sklRF
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
from urllib.request import urlretrieve
import gzip
from cuml import ForestInference
import time
from numba import cuda


In [2]:
print("cudf version: ", cudf.__version__)
print("cuml version: ", cuml.__version__)
print("skl version: ", skl.__version__)

cudf version:  0.10.0a+810.g50a9dc7
cuml version:  0.9.0a1+1074.g42be3ac.dirty
skl version:  0.21.2


## Main benchmarking function

In [3]:
def start_bench(run_cuml, run_skl, skip_test, estimator_array, stream_array, depth_array, csv_path, X_train, y_train, X_train_np, y_train_np, X_test_np, y_test_np):
    results = []
    for n_estimators in estimator_array:
        for n_streams in stream_array:
            for max_depth in depth_array:
                # cuml Random Forest params
                cu_rf_params = {
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'n_bins': 16,
                    'split_algo': 1,
                    'n_streams': n_streams
                }

                cu_fit_time = 0
                skl_fit_time = 0
                acc_score_cuml = 0
                acc_score_skl = 0

                if run_cuml:
                    print("====>cuml====")
                    cuml_params = cu_rf_params.copy()
                    print("    cuml params: ", str(cuml_params))
                    cu_rf = cuRF(**cu_rf_params)
                    print("    cuml model: ", str(cu_rf.get_params()))

                    t0 = time.time()
                    cu_rf.fit(X_train, y_train)
                    cu_fit_time = time.time() - t0

                    print("    cuml fits RF: ", cu_fit_time)

                    if not skip_test:
                        # use a subset of test data to inference 
                        cu_rf_predicted = cu_rf.predict(X_test_np[:1000, :])
                        acc_score_cuml = accuracy_score(cu_rf_predicted, y_test_np[:1000])
                        print("    cuml total time: ", time.time() - t0)
                        print("    cuml acc: ", acc_score_cuml)

                if run_skl and n_streams == 8:
                    print("====>sklearn====")
                    sk_params = cu_rf_params.copy()
                    print("    skl params: ", str(cuml_params))

                    sk_params['n_jobs'] = -1
                    del sk_params['n_bins']
                    del sk_params['split_algo']
                    if 'n_streams' in sk_params:
                        del sk_params['n_streams']                
                    rfc = sklRF(**sk_params)

                    t0 = time.time()
                    rfc.fit(X_train_np, y_train_np)
                    skl_fit_time = time.time() - t0

                    print("    skl fits RF: ", skl_fit_time)
                    
                    if not skip_test:
                        skl_predicted = rfc.predict(X_test_np[:1000, :])
                        acc_score_skl = accuracy_score(skl_predicted, y_test_np[:1000])                
                        print("    skl total time: ", time.time() - t0)
                        print("    skl acc: ", acc_score_skl)

                pd.set_option('display.max_colwidth', 300)
                results.append(dict(cu_fit_time=cu_fit_time, acc_score_cuml=acc_score_cuml, skl_fit_time=skl_fit_time, acc_score_skl=acc_score_skl))
                df = pd.DataFrame(results)
                print(df.to_string())
                df.to_csv(csv_path, mode='a')

## Higgs

In [4]:
def download_higgs(compressed_filepath, decompressed_filepath):
    higgs_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz'
    if not os.path.isfile(compressed_filepath):
        urlretrieve(higgs_url, compressed_filepath)
    if not os.path.isfile(decompressed_filepath):
        cf = gzip.GzipFile(compressed_filepath)
        with open(decompressed_filepath, 'wb') as df:
            df.write(cf.read())

In [5]:
# compressed_filepath = 'HIGGS.csv.gz' # Set this as path for gzipped Higgs data file, if you already have
decompressed_filepath = './HIGGS.csv' # Set this as path for decompressed Higgs data file, if you already have
# download_higgs(compressed_filepath, decompressed_filepath)

col_names = ['label'] + ["col-{}".format(i) for i in range(2, 30)] # Assign column names
dtypes_ls = ['int32'] + ['float32' for _ in range(2, 30)] # Assign dtypes to each column
data = cudf.read_csv(decompressed_filepath, names=col_names, dtype=dtypes_ls)

y_cudf = data['label']
X_cudf = data.drop('label')
from cuml.preprocessing.model_selection import train_test_split
# train_size is the ratio of the entire dataset to be split into training data
X_train, X_test, y_train, y_test = train_test_split(X_cudf, y_cudf, train_size=0.80)

print("Shape of the training data : ", X_train.shape)
print("Shape of the ground truth data used for training : ", y_train.shape)
print("Shape of the testing data : ", X_test.shape)
print("Shape of the ground truth data used for testing : ",y_test.shape)

X_train_np = X_train.as_matrix()
y_train_np = y_train.to_array()
X_test_np = X_test.as_matrix()
y_test_np = y_test.to_array()

Shape of the training data :  (8800000, 28)
Shape of the ground truth data used for training :  (8800000,)
Shape of the testing data :  (2200000, 28)
Shape of the ground truth data used for testing :  (2200000,)


In [6]:
data.head().to_pandas()

,label,col-2,col-3,col-4,col-5,col-6,col-7,col-8,col-9,col-10,...,col-20,col-21,col-22,col-23,col-24,col-25,col-26,col-27,col-28,col-29
0,1,0.869293,-0.635082,0.225690,0.327470,-0.689993,0.754202,-0.248573,-1.092064,0.000000,...,-0.010455,-0.045767,3.101961,1.353760,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,1,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,...,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
2,1,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282323,1.381664,0.000000,...,1.128848,0.900461,0.000000,0.909753,1.108331,0.985692,0.951331,0.803251,0.865924,0.780118
3,0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,...,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
4,1,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,...,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808486


In [7]:
estimator_array = [100, 500, 1000]
stream_array = [8, 10]
depth_array = [8, 12, 16]
run_cuml = True
run_skl = True 
skip_test = True 
csv_path = './rf_bench_results/rf_bench_higgs.csv'

start_bench(run_cuml, run_skl, skip_test, estimator_array, stream_array, depth_array, csv_path, X_train, y_train, X_train_np, y_train_np, X_test_np, y_test_np)

====>cuml====
    cuml params:  {'n_estimators': 100, 'max_depth': 8, 'n_bins': 16, 'split_algo': 1, 'n_streams': 8}
    cuml model:  {'n_estimators': 100, 'max_depth': 8, 'handle': <cuml.common.handle.Handle object at 0x7f18241dad20>, 'max_features': 1.0, 'n_bins': 16, 'split_algo': 1, 'split_criterion': 0, 'min_rows_per_node': 2, 'bootstrap': True, 'bootstrap_features': False, 'verbose': False, 'rows_sample': 1.0, 'max_leaves': -1, 'quantile_per_tree': False}


/gpfs/fs1/rlan/anaconda3/envs/cuml_rf_bench/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Using NumPy for number of class detection,install CuPy for faster processing.


    cuml fits RF:  10.489315032958984
====>sklearn====
    skl params:  {'n_estimators': 100, 'max_depth': 8, 'n_bins': 16, 'split_algo': 1, 'n_streams': 8}
    skl fits RF:  199.35952138900757
   acc_score_cuml  acc_score_skl  cu_fit_time  skl_fit_time
0               0              0    10.489315    199.359521
====>cuml====
    cuml params:  {'n_estimators': 100, 'max_depth': 12, 'n_bins': 16, 'split_algo': 1, 'n_streams': 8}
    cuml model:  {'n_estimators': 100, 'max_depth': 12, 'handle': <cuml.common.handle.Handle object at 0x7f1821e59c30>, 'max_features': 1.0, 'n_bins': 16, 'split_algo': 1, 'split_criterion': 0, 'min_rows_per_node': 2, 'bootstrap': True, 'bootstrap_features': False, 'verbose': False, 'rows_sample': 1.0, 'max_leaves': -1, 'quantile_per_tree': False}


/gpfs/fs1/rlan/anaconda3/envs/cuml_rf_bench/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Using NumPy for number of class detection,install CuPy for faster processing.


    cuml fits RF:  21.808729887008667
====>sklearn====
    skl params:  {'n_estimators': 100, 'max_depth': 12, 'n_bins': 16, 'split_algo': 1, 'n_streams': 8}
    skl fits RF:  274.1888270378113
   acc_score_cuml  acc_score_skl  cu_fit_time  skl_fit_time
0               0              0    10.489315    199.359521
1               0              0    21.808730    274.188827
====>cuml====
    cuml params:  {'n_estimators': 100, 'max_depth': 16, 'n_bins': 16, 'split_algo': 1, 'n_streams': 8}
    cuml model:  {'n_estimators': 100, 'max_depth': 16, 'handle': <cuml.common.handle.Handle object at 0x7f1821e59ed0>, 'max_features': 1.0, 'n_bins': 16, 'split_algo': 1, 'split_criterion': 0, 'min_rows_per_node': 2, 'bootstrap': True, 'bootstrap_features': False, 'verbose': False, 'rows_sample': 1.0, 'max_leaves': -1, 'quantile_per_tree': False}


...

    cuml fits RF:  200.61848092079163
====>sklearn====
    skl params:  {'n_estimators': 1000, 'max_depth': 12, 'n_bins': 16, 'split_algo': 1, 'n_streams': 8}
    skl fits RF:  2019.8885657787323
    acc_score_cuml  acc_score_skl  cu_fit_time  skl_fit_time
0                0              0    10.489315    199.359521
1                0              0    21.808730    274.188827
2                0              0    52.855419    323.933944
3                0              0    10.540802      0.000000
4                0              0    21.862016      0.000000
5                0              0    52.984268      0.000000
6                0              0    45.196291    762.008556
7                0              0   101.434557   1057.250594
8                0              0   252.694696   1266.536006
9                0              0    45.207937      0.000000
10               0              0   101.451268      0.000000
11               0              0   253.021085      0.000000
12         

/gpfs/fs1/rlan/anaconda3/envs/cuml_rf_bench/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Using NumPy for number of class detection,install CuPy for faster processing.


    cuml fits RF:  503.84205627441406
====>sklearn====
    skl params:  {'n_estimators': 1000, 'max_depth': 16, 'n_bins': 16, 'split_algo': 1, 'n_streams': 8}
    skl fits RF:  2440.88609957695
    acc_score_cuml  acc_score_skl  cu_fit_time  skl_fit_time
0                0              0    10.489315    199.359521
1                0              0    21.808730    274.188827
2                0              0    52.855419    323.933944
3                0              0    10.540802      0.000000
4                0              0    21.862016      0.000000
5                0              0    52.984268      0.000000
6                0              0    45.196291    762.008556
7                0              0   101.434557   1057.250594
8                0              0   252.694696   1266.536006
9                0              0    45.207937      0.000000
10               0              0   101.451268      0.000000
11               0              0   253.021085      0.000000
12           

/gpfs/fs1/rlan/anaconda3/envs/cuml_rf_bench/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Using NumPy for number of class detection,install CuPy for faster processing.


    cuml fits RF:  88.56517386436462
    acc_score_cuml  acc_score_skl  cu_fit_time  skl_fit_time
0                0              0    10.489315    199.359521
1                0              0    21.808730    274.188827
2                0              0    52.855419    323.933944
3                0              0    10.540802      0.000000
4                0              0    21.862016      0.000000
5                0              0    52.984268      0.000000
6                0              0    45.196291    762.008556
7                0              0   101.434557   1057.250594
8                0              0   252.694696   1266.536006
9                0              0    45.207937      0.000000
10               0              0   101.451268      0.000000
11               0              0   253.021085      0.000000
12               0              0    88.535205   1450.867585
13               0              0   200.618481   2019.888566
14               0              0   503.842056  

## Airline

In [7]:
from datasets import prepare_dataset

data = prepare_dataset('./data/', 'airline', 115000000)

data.X_train = data.X_train.astype(np.float32)
data.X_test = data.X_test.astype(np.float32)
data.y_train = data.y_train.astype(np.int32)
data.y_test = data.y_test.astype(np.int32)

X_train_np = data.X_train.to_numpy()
X_test_np = data.X_test.to_numpy()
y_train_np = data.y_train.to_numpy()
y_test_np = data.y_test.to_numpy()

X_train_df = cudf.from_pandas(data.X_train)
y_train_df = cudf.from_pandas(data.y_train)

print("NP Shape of the training data : ", X_train_np.shape)
print("NP Shape of the ground truth data used for training : ", y_train_np.shape)
print("NP Shape of the testing data : ", X_test_np.shape)
print("NP Shape of the ground truth data used for testing : ", y_test_np.shape)
print("DF Shape of the training data : ", X_train_df.shape)
print("DF Shape of the ground truth data used for training : ", y_train_df.shape)

NP Shape of the training data :  (92000000, 13)
NP Shape of the ground truth data used for training :  (92000000,)
NP Shape of the testing data :  (23000000, 13)
NP Shape of the ground truth data used for testing :  (23000000,)
DF Shape of the training data :  (92000000, 13)
DF Shape of the ground truth data used for training :  (92000000,)


In [10]:
estimator_array = [100, 500, 1000]
stream_array = [8, 10]
depth_array = [8, 12, 16]
run_cuml = True
run_skl = True 
skip_test = True 
csv_path = './rf_bench_results/rf_bench_airline.csv'

# CudaAPIError: [1] Call to cuMemcpyHtoD results in CUDA_ERROR_INVALID_VALUE
# got this error even with numpy array with input 
start_bench(run_cuml, run_skl, skip_test, estimator_array, stream_array, depth_array, csv_path, X_train_np, y_train_np, X_train_np, X_test_np, X_test_np, y_test_np)


ERROR:Call to cuMemcpyHtoD results in CUDA_ERROR_INVALID_VALUE


====>cuml====
    cuml params:  {'n_estimators': 100, 'max_depth': 8, 'n_bins': 16, 'split_algo': 1, 'n_streams': 8}
    cuml model:  {'n_estimators': 100, 'max_depth': 8, 'handle': <cuml.common.handle.Handle object at 0x7ff978f7cbe8>, 'max_features': 1.0, 'n_bins': 16, 'split_algo': 1, 'split_criterion': 0, 'min_rows_per_node': 2, 'bootstrap': True, 'bootstrap_features': False, 'verbose': False, 'rows_sample': 1.0, 'max_leaves': -1, 'quantile_per_tree': False}


CudaAPIError: [1] Call to cuMemcpyHtoD results in CUDA_ERROR_INVALID_VALUE

## Epsilon

In [5]:
from datasets import prepare_dataset

data = prepare_dataset('./data/', 'epsilon', 500000)

(400000, 2000)


In [19]:
col_names = ["col-{}".format(i) for i in range(0, 2000)] # Assign column names
X_train_df = pd.DataFrame(data.X_train, columns=col_names, dtype=np.float32)
X_train_df_gpu = cudf.from_pandas(X_train_df)

y_train_df = pd.DataFrame(data.y_train, columns=['label'], dtype=np.int32)
y_train_df_gpu = cudf.from_pandas(y_train_df)

print("NP Shape of the training data : ", data.X_train.shape)
print("NP Shape of the ground truth data used for training : ", data.y_train.shape)
print("NP Shape of the testing data : ", data.X_test.shape)
print("NP Shape of the ground truth data used for testing : ", data.y_test.shape)
print("DF Shape of the training data : ", X_train_df_gpu.shape)
print("DF Shape of the ground truth data used for training : ", y_train_df_gpu.shape)

(400000,)
NP Shape of the training data :  (400000, 2000)
NP Shape of the ground truth data used for training :  (400000,)
NP Shape of the testing data :  (100000, 2000)
NP Shape of the ground truth data used for testing :  (100000,)
DF Shape of the training data :  (400000, 2000)
DF Shape of the ground truth data used for training :  (400000, 1)


In [23]:
estimator_array = [100, 500, 1000]
stream_array = [8, 10]
depth_array = [8, 12, 16]
run_cuml = True
run_skl = True 
skip_test = True 
csv_path = './rf_bench_results/rf_bench_epsilon.csv'

start_bench(run_cuml, run_skl, skip_test, estimator_array, stream_array, depth_array, csv_path, X_train_df_gpu, y_train_df_gpu, data.X_train, data.y_train, data.X_test, data.y_test)



====>cuml====
    cuml params:  {'n_estimators': 100, 'max_depth': 12, 'n_bins': 16, 'split_algo': 1, 'n_streams': 10}
    cuml model:  {'n_estimators': 100, 'max_depth': 12, 'handle': <cuml.common.handle.Handle object at 0x7fd88e5e4d68>, 'max_features': 1.0, 'n_bins': 16, 'split_algo': 1, 'split_criterion': 0, 'min_rows_per_node': 2, 'bootstrap': True, 'bootstrap_features': False, 'verbose': False, 'rows_sample': 1.0, 'max_leaves': -1, 'quantile_per_tree': False}


/gpfs/fs1/rlan/anaconda3/envs/cuml_rf_bench/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Using NumPy for number of class detection,install CuPy for faster processing.


RuntimeError: Exception occured! file=/gpfs/fs1/rlan/rf_bench/cuml/cpp/src_prims/common/cuml_allocator.hpp line=109: FAIL: call='cudaMalloc(&ptr, n)'. Reason:out of memory

Obtained 64 stack frames
#0 in /gpfs/fs1/rlan/anaconda3/envs/cuml_rf_bench/lib/libcuml++.so(_ZN8MLCommon9Exception16collectCallStackEv+0x2b) [0x7fd7af76833b]
#1 in /gpfs/fs1/rlan/anaconda3/envs/cuml_rf_bench/lib/libcuml++.so(_ZN8MLCommon9ExceptionC2ERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE+0x62) [0x7fd7af769082]
#2 in /gpfs/fs1/rlan/anaconda3/envs/cuml_rf_bench/lib/libcuml++.so(_ZN8MLCommon22defaultDeviceAllocator8allocateEmP11CUstream_st+0xc3) [0x7fd7af7692d3]
#3 in /gpfs/fs1/rlan/anaconda3/envs/cuml_rf_bench/lib/libcuml++.so(_ZN8MLCommon11buffer_baseIjNS_15deviceAllocatorEEC2ESt10shared_ptrIS1_EP11CUstream_stm+0x83) [0x7fd7af7cc9b3]
#4 in /gpfs/fs1/rlan/anaconda3/envs/cuml_rf_bench/lib/libcuml++.so(_ZN15TemporaryMemoryIfiE17LevelMemAllocatorEiiiiii+0x8fc) [0x7fd7af7cddec]
#5 in /gpfs/fs1/rlan/anacond

## Bosch

In [16]:
from datasets import prepare_dataset

data = prepare_dataset('./data/', 'bosch', 1184000)

data.X_train = data.X_train.astype(np.float32)
data.X_test = data.X_test.astype(np.float32)
data.y_train = data.y_train.astype(np.int32)
data.y_test = data.y_test.astype(np.int32)

X_train_np = data.X_train.to_numpy()
X_test_np = data.X_test.to_numpy()
y_train_np = data.y_train.to_numpy()
y_test_np = data.y_test.to_numpy()

# OOM 
# X_train_df = cudf.from_pandas(data.X_train)
# y_train_df = cudf.from_pandas(data.y_train)

print("NP Shape of the training data : ", X_train_np.shape)
print("NP Shape of the ground truth data used for training : ", y_train_np.shape)
print("NP Shape of the testing data : ", X_test_np.shape)
print("NP Shape of the ground truth data used for testing : ", y_test_np.shape)
# print("DF Shape of the training data : ", X_train_df.shape)
# print("DF Shape of the ground truth data used for training : ", y_train_df.shape)

NP Shape of the training data :  (946997, 968)
NP Shape of the ground truth data used for training :  (946997,)
NP Shape of the testing data :  (236750, 968)
NP Shape of the ground truth data used for testing :  (236750,)


In [23]:
# X_train contains NaN 
print(np.where(np.isnan(X_train_np)))
# fill with zero 
# otherwise sklearn has error: ValueError: Input contains NaN, infinity or a value too large for dtype('float32').
X_train_np = np.nan_to_num(X_train_np)

# X_test contains NaN 
print(np.where(np.isnan(X_test_np)))
# fill with zero 
# otherwise sklearn has error: ValueError: Input contains NaN, infinity or a value too large for dtype('float32').
X_test_np = np.nan_to_num(X_test_np)

(array([], dtype=int64), array([], dtype=int64))
(array([     0,      0,      0, ..., 236749, 236749, 236749]), array([ 23,  24,  25, ..., 954, 955, 956]))


In [26]:
estimator_array = [100, 500, 1000]
stream_array = [8, 10]
depth_array = [8, 12, 16]
run_cuml = True
run_skl = True 
skip_test = True 
csv_path = './rf_bench_results/rf_bench_bosch.csv'

# use X_train_np, y_train_np for cuml as GPU data frame OOM 
# RuntimeError: Exception occured! file=/gpfs/fs1/rlan/rf_bench/cuml/cpp/src/decisiontree/quantile/quantile.cuh line=110: FAIL: call='cudaGetLastError()'. Reason:out of memory
start_bench(run_cuml, run_skl, skip_test, estimator_array, stream_array, depth_array, csv_path, X_train_np, y_train_np, X_train_np, y_train_np, X_test_np, y_test_np)


====>cuml====
    cuml params:  {'n_estimators': 100, 'max_depth': 8, 'n_bins': 16, 'split_algo': 1, 'n_streams': 8}
    cuml model:  {'n_estimators': 100, 'max_depth': 8, 'handle': <cuml.common.handle.Handle object at 0x7ff99bf16c90>, 'max_features': 1.0, 'n_bins': 16, 'split_algo': 1, 'split_criterion': 0, 'min_rows_per_node': 2, 'bootstrap': True, 'bootstrap_features': False, 'verbose': False, 'rows_sample': 1.0, 'max_leaves': -1, 'quantile_per_tree': False}


/gpfs/fs1/rlan/anaconda3/envs/cuml_rf_bench/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Using NumPy for number of class detection,install CuPy for faster processing.


    cuml fits RF:  68.05904006958008
    cuml total time:  68.62241744995117
    cuml acc:  0.99
====>sklearn====
    skl params:  {'n_estimators': 100, 'max_depth': 8, 'n_bins': 16, 'split_algo': 1, 'n_streams': 8}
    skl fits RF:  22.116517305374146
    skl total time:  22.73055362701416
    skl acc:  0.99
   acc_score_cuml  acc_score_skl  cu_fit_time  skl_fit_time
0            0.99           0.99     68.05904     22.116517
====>cuml====
    cuml params:  {'n_estimators': 100, 'max_depth': 8, 'n_bins': 16, 'split_algo': 1, 'n_streams': 10}
    cuml model:  {'n_estimators': 100, 'max_depth': 8, 'handle': <cuml.common.handle.Handle object at 0x7ff99bf16db0>, 'max_features': 1.0, 'n_bins': 16, 'split_algo': 1, 'split_criterion': 0, 'min_rows_per_node': 2, 'bootstrap': True, 'bootstrap_features': False, 'verbose': False, 'rows_sample': 1.0, 'max_leaves': -1, 'quantile_per_tree': False}


/gpfs/fs1/rlan/anaconda3/envs/cuml_rf_bench/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Using NumPy for number of class detection,install CuPy for faster processing.


    cuml fits RF:  68.04239845275879
    cuml total time:  68.54566097259521
    cuml acc:  0.99
   acc_score_cuml  acc_score_skl  cu_fit_time  skl_fit_time
0            0.99           0.99    68.059040     22.116517
1            0.99           0.00    68.042398      0.000000
====>cuml====
    cuml params:  {'n_estimators': 500, 'max_depth': 8, 'n_bins': 16, 'split_algo': 1, 'n_streams': 8}
    cuml model:  {'n_estimators': 500, 'max_depth': 8, 'handle': <cuml.common.handle.Handle object at 0x7ff99bf16f90>, 'max_features': 1.0, 'n_bins': 16, 'split_algo': 1, 'split_criterion': 0, 'min_rows_per_node': 2, 'bootstrap': True, 'bootstrap_features': False, 'verbose': False, 'rows_sample': 1.0, 'max_leaves': -1, 'quantile_per_tree': False}
    cuml fits RF:  316.58206844329834
    cuml total time:  319.22181463241577
    cuml acc:  0.99
====>sklearn====
    skl params:  {'n_estimators': 500, 'max_depth': 8, 'n_bins': 16, 'split_algo': 1, 'n_streams': 8}
    skl fits RF:  89.02854681015015
   

/gpfs/fs1/rlan/anaconda3/envs/cuml_rf_bench/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Using NumPy for number of class detection,install CuPy for faster processing.


    cuml fits RF:  316.32362055778503
    cuml total time:  318.9319658279419
    cuml acc:  0.99
   acc_score_cuml  acc_score_skl  cu_fit_time  skl_fit_time
0            0.99           0.99    68.059040     22.116517
1            0.99           0.00    68.042398      0.000000
2            0.99           0.99   316.582068     89.028547
3            0.99           0.00   316.323621      0.000000
====>cuml====
    cuml params:  {'n_estimators': 1000, 'max_depth': 8, 'n_bins': 16, 'split_algo': 1, 'n_streams': 8}
    cuml model:  {'n_estimators': 1000, 'max_depth': 8, 'handle': <cuml.common.handle.Handle object at 0x7ff99bf363a8>, 'max_features': 1.0, 'n_bins': 16, 'split_algo': 1, 'split_criterion': 0, 'min_rows_per_node': 2, 'bootstrap': True, 'bootstrap_features': False, 'verbose': False, 'rows_sample': 1.0, 'max_leaves': -1, 'quantile_per_tree': False}
    cuml fits RF:  626.5133574008942
    cuml total time:  631.721670627594
    cuml acc:  0.99
====>sklearn====
    skl params:  {'n_

/gpfs/fs1/rlan/anaconda3/envs/cuml_rf_bench/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Using NumPy for number of class detection,install CuPy for faster processing.


    cuml fits RF:  625.6281564235687
    cuml total time:  630.6998109817505
    cuml acc:  0.99
   acc_score_cuml  acc_score_skl  cu_fit_time  skl_fit_time
0            0.99           0.99    68.059040     22.116517
1            0.99           0.00    68.042398      0.000000
2            0.99           0.99   316.582068     89.028547
3            0.99           0.00   316.323621      0.000000
4            0.99           0.99   626.513357    170.811307
5            0.99           0.00   625.628156      0.000000
